## 1. 克隆代码库

In [ ]:
# 克隆代码仓库
!git clone https://github.com/Wu-XiaoTian/myLLM.git
%cd myLLM

# 验证文件结构
!ls -la utils/

## 2. 安装依赖

In [ ]:
# 安装必要的依赖包
!pip install -q numpy scipy torch diffusers transformers opencv-python opencv-contrib-python
!pip install -q inflect easydict accelerate gradio pydantic
!pip install -q openai  # 用于兼容 OpenAI API 调用格式

## 3. 设置 API Key

In [ ]:
# 设置火山引擎 ARK API Key 并配置 Python 路径
import os
import sys

# 设置 API Key
os.environ['ARK_API_KEY'] = '29d9f392-5151-47e4-b1f6-c007d69f4ae9'

# 添加当前目录到 Python 路径（重要！确保可以导入项目模块）
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

print("✓ API Key 已设置")
print(f"✓ Python 路径已配置: {current_dir}")


## 4. 测试 API 连接

In [ ]:
# 测试火山引擎豆包 API
from openai import OpenAI
import os

api_key = os.getenv('ARK_API_KEY')

client = OpenAI(
    base_url="https://ark.cn-beijing.volces.com/api/v3",
    api_key=api_key,
)

# 测试简单的文本对话
try:
    response = client.chat.completions.create(
        model="doubao-seed-1-6-251015",
        messages=[
            {
                "role": "user",
                "content": "Hello, can you help me?"
            }
        ]
    )
    print("API 测试成功！")
    print(f"回复: {response.choices[0].message.content}")
except Exception as e:
    print(f"API 测试失败: {e}")

## 5. 查看配置文件

In [ ]:
# 验证 API key 配置
!cat utils/api_key.py

## 5.5 配置 Python 路径

确保 Python 可以正确导入项目模块。

In [ ]:
# 添加当前目录到 Python 路径，确保能导入 utils 模块
import sys
import os

# 获取当前工作目录
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
    print(f"✓ 已添加 {current_dir} 到 Python 路径")

# 验证可以导入 utils 模块
try:
    from utils.api_key import api_key
    print("✓ 成功导入 utils.api_key 模块")
    print(f"  API Key 前缀: {api_key[:10]}...")
except Exception as e:
    print(f"✗ 导入失败: {e}")
    print(f"  当前目录: {os.getcwd()}")
    print(f"  sys.path: {sys.path[:3]}")


## 6. 运行基本示例

使用豆包模型生成布局。注意：需要指定 `doubao-seed-1-6-251015` 作为模型名称。

In [ ]:
# 确保 Python 路径已配置(如果之前没运行第 3 步,这里会自动配置)
import sys
import os
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())
    print(f"✓ 已配置 Python 路径: {os.getcwd()}")

# 导入必要的模块
from utils.llm import get_llm_kwargs, get_parsed_layout

# 设置模型和模板版本
model = "doubao-seed-1-6-251015"
template_version = "v0.1"  # 根据实际情况调整

# 获取 LLM 配置
model_name, llm_kwargs = get_llm_kwargs(model, template_version)

# 测试提示词
test_prompt = "A cat sitting on a table next to a red apple"

print(f"使用模型: {model_name}")
print(f"测试提示词: {test_prompt}")
print("\n正在生成布局...")

try:
    gen_boxes, bg_prompt, neg_prompt = get_parsed_layout(test_prompt, llm_kwargs)
    print("\n✓ 生成成功!")
    print(f"生成的边界框数量: {len(gen_boxes)}")
    print(f"背景提示词: {bg_prompt}")
except Exception as e:
    print(f"\n✗ 生成失败: {e}")
    import traceback
    traceback.print_exc()


## 7. 完整图像生成示例

运行完整的图像生成流程（需要 GPU）。

In [ ]:
# 检查 GPU 是否可用
import torch
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 设备: {torch.cuda.get_device_name(0)}")
    print(f"GPU 内存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

In [ ]:
# 运行完整的生成流程
# 注意：这需要较大的 GPU 内存

# 示例命令（根据你的需求调整参数）
!python generate.py \
    --prompt "A beautiful sunset over mountains with a lake in the foreground" \
    --model doubao-seed-1-6-251015 \
    --template-version v0.1 \
    --output-dir ./outputs

## 8. 可视化结果

In [ ]:
# 显示生成的图像
from IPython.display import Image, display
import os

output_dir = "./outputs"
if os.path.exists(output_dir):
    images = [f for f in os.listdir(output_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    for img in images:
        print(f"显示图像: {img}")
        display(Image(filename=os.path.join(output_dir, img)))
else:
    print("输出目录不存在")

## 注意事项

1. **API Key 安全**: 请妥善保管你的 API Key，不要在公开的笔记本中硬编码
2. **GPU 要求**: 图像生成需要 GPU，在 Colab 中请选择 GPU 运行时
3. **模型兼容性**: 豆包模型可能与 OpenAI 的接口有些差异，如遇问题请检查 API 文档
4. **网络连接**: 确保 Colab 可以访问火山引擎的 API 端点
5. **依赖版本**: 某些包的版本可能需要调整以确保兼容性